# WGAN-GP overriding `Model.train_step`

**Author:** [A_K_Nain](https://twitter.com/A_K_Nain)<br>
**Date created:** 2020/05/9<br>
**Last modified:** 2020/05/9<br>
**Description:** Implementation of Wasserstein GAN with Gradient Penalty.

I am modifying this to input my data

## Wasserstein GAN (WGAN) with Gradient Penalty (GP)

The original [Wasserstein GAN](https://arxiv.org/abs/1701.07875) leverages the
Wasserstein distance to produce a value function that has better theoretical
properties than the value function used in the original GAN paper. WGAN requires
that the discriminator (aka the critic) lie within the space of 1-Lipschitz
functions. The authors proposed the idea of weight clipping to achieve this
constraint. Though weight clipping works, it can be a problematic way to enforce
1-Lipschitz constraint and can cause undesirable behavior, e.g. a very deep WGAN
discriminator (critic) often fails to converge.

The [WGAN-GP](https://arxiv.org/pdf/1704.00028.pdf) method proposes an
alternative to weight clipping to ensure smooth training. Instead of clipping
the weights, the authors proposed a "gradient penalty" by adding a loss term
that keeps the L2 norm of the discriminator gradients close to 1.


## Setup


In [14]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd


In [15]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = "../"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
taxi = pd.read_csv(f"{proj_dir}data_clean/taxi.csv")
column_names = taxi.columns
taxi = taxi.to_numpy()

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
std_scaler = scaler.fit(taxi)
taxi = std_scaler.transform(taxi)

In [18]:
n_sq_df = np.floor_divide(taxi.shape[0], taxi.shape[1])

In [19]:
arr = np.arange(taxi.shape[1] * n_sq_df)
np.random.shuffle(arr)
taxi = taxi[arr, :, ].reshape(n_sq_df, 28, 28, 1)

In [20]:
data_SHAPE = (28, 28, 1)
BATCH_SIZE = 250

# Size of the noise vector
noise_dim = 128


## Create the discriminator (the critic in the original WGAN)

The samples in the dataset have a (28, 28, 1) shape. Because we will be
using strided convolutions, this can result in a shape with odd dimensions.
For example,
`(28, 28) -> Conv_s2 -> (14, 14) -> Conv_s2 -> (7, 7) -> Conv_s2 ->(3, 3)`.

While peforming upsampling in the generator part of the network, we won't get 
the same input shape as the original datas if we aren't careful. To avoid this,
we will do something much simpler:
- In the discriminator: "zero pad" the input to change the shape to `(32, 32, 1)`
for each sample; and
- Ihe generator: crop the final output to match the shape with input shape.

In [21]:

def conv_block(
    x,
    filters,
    activation,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="same",
    use_bias=True,
    use_bn=False,
    use_dropout=False,
    drop_value=0.5,
):
    x = layers.Conv2D(
        filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias
    )(x)
    if use_bn:
        x = layers.BatchNormalization()(x)
    x = activation(x)
    if use_dropout:
        x = layers.Dropout(drop_value)(x)
    return x


def get_discriminator_model():
    data_input = layers.Input(shape=data_SHAPE)
    # Zero pad the input to make the input datas size to (32, 32, 1).
    x = layers.ZeroPadding2D((2, 2))(data_input)
    x = conv_block(
        x,
        64,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        use_bias=True,
        activation=layers.LeakyReLU(0.2),
        use_dropout=False,
        drop_value=0.3,
    )
    x = conv_block(
        x,
        128,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=True,
        drop_value=0.3,
    )
    x = conv_block(
        x,
        256,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=True,
        drop_value=0.3,
    )
    x = conv_block(
        x,
        512,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=False,
        drop_value=0.3,
    )

    x = layers.Flatten()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1)(x)

    d_model = keras.models.Model(data_input, x, name="discriminator")
    return d_model


d_model = get_discriminator_model()
d_model.summary()


Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 32, 32, 1)        0         
 ng2D)                                                           
                                                                 
 conv2d_7 (Conv2D)           (None, 16, 16, 64)        1664      
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 16, 16, 64)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 8, 8, 128)         204928    
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 8, 8, 128)         0         
                                                     

## Create the generator


In [22]:

def upsample_block(
    x,
    filters,
    activation,
    kernel_size=(3, 3),
    strides=(1, 1),
    up_size=(2, 2),
    padding="same",
    use_bn=False,
    use_bias=True,
    use_dropout=False,
    drop_value=0.3,
):
    x = layers.UpSampling2D(up_size)(x)
    x = layers.Conv2D(
        filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias
    )(x)

    if use_bn:
        x = layers.BatchNormalization()(x)

    if activation:
        x = activation(x)
    if use_dropout:
        x = layers.Dropout(drop_value)(x)
    return x


def get_generator_model():
    noise = layers.Input(shape=(noise_dim,))
    x = layers.Dense(4 * 4 * 256, use_bias=False)(noise)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Reshape((4, 4, 256))(x)
    x = upsample_block(
        x,
        128,
        layers.LeakyReLU(0.2),
        strides=(1, 1),
        use_bias=False,
        use_bn=True,
        padding="same",
        use_dropout=False,
    )
    x = upsample_block(
        x,
        64,
        layers.LeakyReLU(0.2),
        strides=(1, 1),
        use_bias=False,
        use_bn=True,
        padding="same",
        use_dropout=False,
    )
    x = upsample_block(
        x, 1, layers.Activation("tanh"), strides=(1, 1), use_bias=False, use_bn=True
    )
    # At this point, we have an output which has the same shape as the input, (32, 32, 1).
    # We will use a Cropping2D layer to make it (28, 28, 1).
    x = layers.Cropping2D((2, 2))(x)

    g_model = keras.models.Model(noise, x, name="generator")
    return g_model


g_model = get_generator_model()
g_model.summary()


Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128)]             0         
                                                                 
 dense_3 (Dense)             (None, 4096)              524288    
                                                                 
 batch_normalization_4 (Batc  (None, 4096)             16384     
 hNormalization)                                                 
                                                                 
 leaky_re_lu_11 (LeakyReLU)  (None, 4096)              0         
                                                                 
 reshape_1 (Reshape)         (None, 4, 4, 256)         0         
                                                                 
 up_sampling2d_3 (UpSampling  (None, 8, 8, 256)        0         
 2D)                                                     

## Create the WGAN-GP model

Now that we have defined our generator and discriminator, it's time to implement
the WGAN-GP model. We will also override the `train_step` for training.

In [23]:
class WGAN(keras.Model):
    def __init__(
        self,
        discriminator,
        generator,
        latent_dim,
        discriminator_extra_steps=3,
        gp_weight=10.0,
    ):
        super(WGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_steps = discriminator_extra_steps
        self.gp_weight = gp_weight

    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn):
        super(WGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.g_loss_fn = g_loss_fn

    def gradient_penalty(self, batch_size, real_datas, fake_datas):
        """ Calculates the gradient penalty.

        This loss is calculated on an interpolated data
        and added to the discriminator loss.
        """
        # Get the interpolated data
        alpha = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake_datas - real_datas
        interpolated = real_datas + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            # 1. Get the discriminator output for this interpolated data.
            pred = self.discriminator(interpolated, training=True)

        # 2. Calculate the gradients w.r.t to this interpolated data.
        grads = gp_tape.gradient(pred, [interpolated])[0]
        # 3. Calculate the norm of the gradients.
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    def train_step(self, real_datas):
        if isinstance(real_datas, tuple):
            real_datas = real_datas[0]

        # Get the batch size
        batch_size = tf.shape(real_datas)[0]

        # For each batch, we are going to perform the
        # following steps as laid out in the original paper:
        # 1. Train the generator and get the generator loss
        # 2. Train the discriminator and get the discriminator loss
        # 3. Calculate the gradient penalty
        # 4. Multiply this gradient penalty with a constant weight factor
        # 5. Add the gradient penalty to the discriminator loss
        # 6. Return the generator and discriminator losses as a loss dictionary

        # Train the discriminator first. The original paper recommends training
        # the discriminator for `x` more steps (typically 5) as compared to
        # one step of the generator. Here we will train it for 3 extra steps
        # as compared to 5 to reduce the training time.
        for i in range(self.d_steps):
            # Get the latent vector
            random_latent_vectors = tf.random.normal(
                shape=(batch_size, self.latent_dim)
            )
            with tf.GradientTape() as tape:
                # Generate fake datas from the latent vector
                fake_datas = self.generator(random_latent_vectors, training=True)
                # Get the logits for the fake datas
                fake_logits = self.discriminator(fake_datas, training=True)
                # Get the logits for the real datas
                real_logits = self.discriminator(real_datas, training=True)

                # Calculate the discriminator loss using the fake and real data logits
                d_cost = self.d_loss_fn(real_data=real_logits, fake_data=fake_logits)
                # Calculate the gradient penalty
                gp = self.gradient_penalty(batch_size, real_datas, fake_datas)
                # Add the gradient penalty to the original discriminator loss
                d_loss = d_cost + gp * self.gp_weight

            # Get the gradients w.r.t the discriminator loss
            d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
            # Update the weights of the discriminator using the discriminator optimizer
            self.d_optimizer.apply_gradients(
                zip(d_gradient, self.discriminator.trainable_variables)
            )

        # Train the generator
        # Get the latent vector
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        with tf.GradientTape() as tape:
            # Generate fake datas using the generator
            generated_datas = self.generator(random_latent_vectors, training=True)
            # Get the discriminator logits for fake datas
            gen_data_logits = self.discriminator(generated_datas, training=True)
            # Calculate the generator loss
            g_loss = self.g_loss_fn(gen_data_logits)

        # Get the gradients w.r.t the generator loss
        gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        # Update the weights of the generator using the generator optimizer
        self.g_optimizer.apply_gradients(
            zip(gen_gradient, self.generator.trainable_variables)
        )
        return {"discriminator_loss": d_loss, "generator_loss": g_loss}

## Create a Keras callback that periodically saves generated datas

In [24]:

class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_data=6, latent_dim=128):
        self.num_data = num_data
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_data, self.latent_dim))
        generated_datas = self.model.generator(random_latent_vectors)
        generated_datas = (generated_datas * 127.5) + 127.5
        import os

        for i in range(self.num_data):
            data = generated_datas[i].numpy()
            # data = keras.preprocessing.data.array_to_data(data)
            output_path = f"{proj_dir}data_generated/conv_wgan_4.csv"
            data = np.reshape(data, (28, 28))
            generated_data = pd.DataFrame(std_scaler.inverse_transform(data),  columns=column_names)
            generated_data['epoch'] = epoch
            generated_data.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)
            # my first concern is going to be the shapes of this. 



## Train the end-to-end model


In [25]:
# Instantiate the optimizer for both networks
# (learning_rate=0.0002, beta_1=0.5 are recommended)
generator_optimizer = keras.optimizers.Adam(
    learning_rate=0.000002, beta_1=0.5, beta_2=0.8
)
discriminator_optimizer = keras.optimizers.Adam(
    learning_rate=0.00002, beta_1=0.5, beta_2=0.8
)

# Define the loss functions for the discriminator,
# which should be (fake_loss - real_loss).
# We will add the gradient penalty later to this loss function.
def discriminator_loss(real_data, fake_data):
    real_loss = tf.reduce_mean(real_data)
    fake_loss = tf.reduce_mean(fake_data)
    return fake_loss - real_loss


# Define the loss functions for the generator.
def generator_loss(fake_data):
    return -tf.reduce_mean(fake_data)


# Set the number of epochs for trainining.
epochs = 500

# Instantiate the customer `GANMonitor` Keras callback.
cbk = GANMonitor(num_data=3, latent_dim=noise_dim)

# Instantiate the WGAN model.
wgan = WGAN(
    discriminator=d_model,
    generator=g_model,
    latent_dim=noise_dim,
    discriminator_extra_steps=3,
)

# Compile the WGAN model.
wgan.compile(
    d_optimizer=discriminator_optimizer,
    g_optimizer=generator_optimizer,
    g_loss_fn=generator_loss,
    d_loss_fn=discriminator_loss,
)



In [ ]:
# Start training the model.
wgan.fit(taxi, batch_size=BATCH_SIZE, epochs=epochs, callbacks=[cbk])

Epoch 1/500
209/209 [==============================] - 40s 169ms/step - discriminator_loss: -21.2408 - generator_loss: 19.5205
Epoch 2/500
209/209 [==============================] - 35s 169ms/step - discriminator_loss: -21.5368 - generator_loss: 14.4047
Epoch 3/500
209/209 [==============================] - 35s 169ms/step - discriminator_loss: -17.2961 - generator_loss: 2.2908
Epoch 4/500
209/209 [==============================] - 35s 169ms/step - discriminator_loss: -13.9056 - generator_loss: -8.1676
Epoch 5/500
209/209 [==============================] - 35s 169ms/step - discriminator_loss: -11.6174 - generator_loss: -14.0027
Epoch 6/500
209/209 [==============================] - 35s 169ms/step - discriminator_loss: -10.0507 - generator_loss: -12.9080
Epoch 7/500
209/209 [==============================] - 35s 169ms/step - discriminator_loss: -9.0769 - generator_loss: -9.8982
Epoch 8/500
209/209 [==============================] - 35s 169ms/step - discriminator_loss: -8.3533 - generator